# Clever API - Fetch All Schools, Teachers, and Students

This notebook fetches:
1. ALL schools in the district
2. ALL teachers for each school
3. ALL students for each school
4. Flattens nested JSON responses
5. Saves everything as CSV files

In [ ]:
# Setup
from pyspark.sql import SparkSession
from pyspark_datasources import rest_api_call, flatten_json_response
import ast

# Initialize Spark
spark = SparkSession.builder \
    .appName("CleverAPIFetchAll") \
    .master("local[*]") \
    .getOrCreate()

# Your Clever API token (Chess4Life Dev Sandbox)
CLEVER_TOKEN = ""

# Create dummy input for GET requests
dummy_input = spark.createDataFrame([{"placeholder": "dummy"}])

print("✓ Setup complete!")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/24 19:02:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✓ Setup complete!


## Step 1: Fetch ALL Schools

In [2]:
print("Fetching ALL schools...")

# Use rest_api_call - the recommended PySpark Data Source approach
schools_response = rest_api_call(
    dummy_input,
    url="https://api.clever.com/v3.0/schools?limit=10000",
    method="GET",
    authType="Bearer",
    oauthToken=CLEVER_TOKEN,
    queryType="inline",
    partitions="1"
)

# Parse the output column to extract school data
school_output = schools_response.select("output").first()["output"]
school_data = ast.literal_eval(school_output)

# Extract schools
all_schools = []
for school in school_data.get("data", []):
    school_info = school.get("data", {})
    all_schools.append({
        "school_id": school_info.get("id"),
        "school_name": school_info.get("name"),
    })

print(f"✓ Fetched {len(all_schools)} schools")

# Convert to DataFrame - this will be our input for teachers and students
schools_df = spark.createDataFrame(all_schools)
schools_df.show(10, truncate=False)
print(f"Total schools: {schools_df.count()}")

Fetching ALL schools...


/Users/yipman/Downloads/github/pyspark-data-sources/.venv/lib/python3.14/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py:161: DeprecationWarning: This process (pid=9919) is multi-threaded, use of fork() may lead to deadlocks in the child.


✓ Fetched 241 schools
+------------------------+-------------+
|school_id               |school_name  |
+------------------------+-------------+
|5940d254203e37907e0000f0|Slothgreat   |
|5940d254203e37907e0000f1|Vultureroad  |
|5940d254203e37907e0000f2|Swoopcharm   |
|5940d254203e37907e0000f3|Razorgem     |
|5940d254203e37907e0000f4|Bunnyflicker |
|5940d254203e37907e0000f5|Spearchestnut|
|5940d254203e37907e0000f6|Hoofrift     |
|5940d254203e37907e0000f7|Elfneon      |
|5940d254203e37907e0000f8|Cowlgiant    |
|5940d254203e37907e0000f9|Bunnybloom   |
+------------------------+-------------+
only showing top 10 rows
Total schools: 241


## Step 2: Fetch ALL Teachers for ALL Schools

In [3]:
print("Fetching ALL teachers using schools DataFrame...")

# Use schools_df as input - rest_api_call will make one API call per school
teachers_response = rest_api_call(
    schools_df,
    url="https://api.clever.com/v3.0/schools/{school_id}/users?role=teacher&limit=10000",
    method="GET",
    authType="Bearer",
    oauthToken=CLEVER_TOKEN,
    queryType="inline"
)

print(f"✓ Fetched teachers for all schools!")
print(f"Teachers response has {teachers_response.count()} rows")

Fetching ALL teachers using schools DataFrame...


25/10/24 19:02:11 WARN DataSourceManager: The data source rest replaced a previously registered data source.
25/10/24 19:02:31 WARN TaskSetManager: Stage 11 contains a task of very large size (1047 KiB). The maximum recommended task size is 1000 KiB.


✓ Fetched teachers for all schools!
Teachers response has 241 rows


## Step 3: Fetch ALL Students for ALL Schools

In [4]:
print("Fetching ALL students using schools DataFrame...")

# Use schools_df as input - rest_api_call will make one API call per school
students_response = rest_api_call(
    schools_df,
    url="https://api.clever.com/v3.0/schools/{school_id}/users?role=student&limit=10000",
    method="GET",
    authType="Bearer",
    oauthToken=CLEVER_TOKEN,
    queryType="inline"
)

print(f"✓ Fetched students for all schools!")
print(f"Students response has {students_response.count()} rows")

Fetching ALL students using schools DataFrame...


25/10/24 19:02:31 WARN DataSourceManager: The data source rest replaced a previously registered data source.


✓ Fetched students for all schools!
Students response has 241 rows


25/10/24 19:03:01 WARN TaskSetManager: Stage 16 contains a task of very large size (5790 KiB). The maximum recommended task size is 1000 KiB.


## Step 4: Flatten Nested JSON Responses

In [5]:
print("Flattening teachers data...")

# Use flatten_json_response to extract and flatten the nested "data" array
# Clever API returns: {"data": [{"data": {...teacher fields...}}, ...]}
# fully_flatten=True recursively flattens ALL nested dicts into columns (default)
# This converts name:{first, last, middle} -> name_first, name_last, name_middle
teachers_df = flatten_json_response(
    teachers_response,
    json_path="data",           # Extract the "data" array
    flatten_nested_key="data",  # Each item has nested "data" object to flatten
    fully_flatten=True          # Recursively flatten all nested structures
)

print(f"✓ Flattened {teachers_df.count()} individual teachers")
print(f"Columns: {teachers_df.columns}")
teachers_df.show(5, truncate=False)

Flattening teachers data...


25/10/24 19:03:02 WARN TaskSetManager: Stage 19 contains a task of very large size (1047 KiB). The maximum recommended task size is 1000 KiB.


✓ Flattened 8490 individual teachers
Columns: ['created', 'district', 'email', 'id', 'last_modified', 'name_first', 'name_last', 'roles_teacher_credentials_district_username', 'roles_teacher_legacy_id', 'roles_teacher_school', 'roles_teacher_schools', 'roles_teacher_sis_id', 'roles_teacher_state_id', 'roles_teacher_teacher_number', 'roles_teacher_title', 'school_id', 'school_name', 'name_middle']
+------------------------+------------------------+----------------------------------------------------------------+------------------------+------------------------+----------+-----------------------------------+-------------------------------------------+------------------------+------------------------+------------------------+--------------------+----------------------+----------------------------+-------------------+------------------------+-----------+-----------+
|created                 |district                |email                                                           |id       

Exception ignored while finalizing file <_io.BufferedWriter name=5>:
Traceback (most recent call last):
  File "/Users/yipman/Downloads/github/pyspark-data-sources/.venv/lib/python3.14/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 200, in manager
BrokenPipeError: [Errno 32] Broken pipe


In [6]:
print("Flattening students data...")

# Use flatten_json_response to extract and flatten the nested "data" array
# Clever API returns: {"data": [{"data": {...student fields...}}, ...]}
# fully_flatten=True recursively flattens ALL nested dicts into columns (default)
# This converts name:{first, last, middle} -> name_first, name_last, name_middle
# and roles:{student:{...}} -> roles_student_dob, roles_student_gender, etc.
students_df = flatten_json_response(
    students_response,
    json_path="data",           # Extract the "data" array
    flatten_nested_key="data",  # Each item has nested "data" object to flatten
    fully_flatten=True          # Recursively flatten all nested structures
)

print(f"✓ Flattened {students_df.count()} individual students")
print(f"Columns: {students_df.columns}")
students_df.show(5, truncate=False)

Flattening students data...


25/10/24 19:03:04 WARN TaskSetManager: Stage 24 contains a task of very large size (5790 KiB). The maximum recommended task size is 1000 KiB.
25/10/24 19:03:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/10/24 19:03:20 WARN TaskSetManager: Stage 25 contains a task of very large size (2454 KiB). The maximum recommended task size is 1000 KiB.


✓ Flattened 56908 individual students
Columns: ['created', 'district', 'email', 'id', 'last_modified', 'name_first', 'name_last', 'name_middle', 'roles_student_credentials_district_username', 'roles_student_dob', 'roles_student_email', 'roles_student_enrollments', 'roles_student_gender', 'roles_student_grade', 'roles_student_graduation_year', 'roles_student_hispanic_ethnicity', 'roles_student_location_address', 'roles_student_location_city', 'roles_student_location_state', 'roles_student_location_zip', 'roles_student_race', 'roles_student_school', 'roles_student_schools', 'roles_student_sis_id', 'roles_student_state_id', 'roles_student_student_number', 'school_id', 'school_name']
+------------------------+------------------------+-------------------------------------------+------------------------+------------------------+----------+------------------+------------+-------------------------------------------+-----------------+-------------------------------------------+-----------------

25/10/24 19:03:20 WARN TaskSetManager: Stage 28 contains a task of very large size (2454 KiB). The maximum recommended task size is 1000 KiB.
Exception ignored while finalizing file <_io.BufferedWriter name=5>:
Traceback (most recent call last):
  File "/Users/yipman/Downloads/github/pyspark-data-sources/.venv/lib/python3.14/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 200, in manager
BrokenPipeError: [Errno 32] Broken pipe


## Step 5: Save Flattened Data as CSV Files

In [7]:
# Save Schools
schools_output = "output/schools.csv"
schools_df.coalesce(1).write.mode("overwrite").option("header", "true").csv(schools_output)
print(f"✓ Schools saved to {schools_output}")

# Save Teachers (flattened - one row per teacher)
teachers_output = "output/teachers.csv"
teachers_df.coalesce(1).write.mode("overwrite").option("header", "true").csv(teachers_output)
print(f"✓ Teachers saved to {teachers_output}")

# Save Students (flattened - one row per student)
students_output = "output/students.csv"
students_df.coalesce(1).write.mode("overwrite").option("header", "true").csv(students_output)
print(f"✓ Students saved to {students_output}")

print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)
print(f"Schools:  {schools_df.count()}")
print(f"Teachers: {teachers_df.count()}")
print(f"Students: {students_df.count()}")
print("\nAll data has been flattened and saved as CSV files.")
print("Each CSV file contains one row per school/teacher/student.")
print("=" * 60)

✓ Schools saved to output/schools.csv


25/10/24 19:03:20 WARN TaskSetManager: Stage 30 contains a task of very large size (2536 KiB). The maximum recommended task size is 1000 KiB.


✓ Teachers saved to output/teachers.csv


25/10/24 19:03:21 WARN TaskSetManager: Stage 31 contains a task of very large size (19539 KiB). The maximum recommended task size is 1000 KiB.


✓ Students saved to output/students.csv

SUMMARY
Schools:  241
Teachers: 8490
Students: 56908

All data has been flattened and saved as CSV files.
Each CSV file contains one row per school/teacher/student.


25/10/24 19:03:21 WARN TaskSetManager: Stage 38 contains a task of very large size (2454 KiB). The maximum recommended task size is 1000 KiB.
